<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/am_data_processing_risk_high_topics_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective:**

The topics extracted are micro level topics. These are goods for sentiment analysis but not aligned with summarization model. So, the risk of the topic along with the high level topics are also determined after sentiment run.

**Approach:**

The following parameters are added.

Marco level topics: These are G-SIB assesment criteria. The micro level topics classified through key words of these topics.

High Level topics: These are key topics of any financial institutions. For example, asset, liiability, profit, loos etc.

Risk: The micro topics may be risky such as credit score. The topics are classified into 3 risk categories. These are High, medium and Low.

**Process:**

The GPT-4-Turbo model prompted with role 'Financial Analyst' to perform the above classification based on micro topics.

# Section 1: Loading the data

In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the actual file path
file_path = "/content/UBS_questions_sent_output_050325_v1.csv"

# Read the CSV file into a DataFrame
input_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral


In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [ ]:
# Import necessary libraries
import openai
import pandas as pd
import os
import re

# Ensure OpenAI API key is set
openai.api_key = ""  # Set your API key here

# Section 2: Classification of risk

In [ ]:
# Function to classify risk using GPT-4-Turbo
def classify_risk(snippet):
    prompt = f"""You are a financial risk assessment model.
    Analyze the following financial snippet and classify the risk level as High, Medium, or Low.
    Only return one of these three words: High, Medium, or Low.

    Financial Snippet: "{snippet}"

    Risk Level:"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error processing snippet: {e}")
        return "Unknown"


In [ ]:
# Apply function to classify risk
input_df["Risk"] = input_df["Snippet"].astype(str).apply(classify_risk)

# Display first few rows of the updated DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent,Risk
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral,Low
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral,High
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral,High
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral,High
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral,High


# Section 3: Classification according to G-SIB crieria

In [ ]:
# Define the topics and associated keywords
topics_keywords = {
    "Cross-jurisdictional claims": "foreign claims, international claims, cross-border claims, local claims in foreign currency, BIS consolidated banking statistics",
    "Cross-jurisdictional liabilities": "foreign liabilities, related offices, BIS consolidated banking statistics, international banking market, local liabilities in local currency",
    "Total exposure": "total assets, total liabilities, risk-weighted assets, leverage ratio, Basel III, capital adequacy, bank exposures",
    "Intra-financial system assets": "lending to financial institutions, securities holdings, reverse repurchase agreements, securities lending, OTC derivatives assets",
    "Intra-financial system liabilities": "deposits by financial institutions, securities issued, repurchase agreements, securities borrowing, OTC derivatives liabilities",
    "Wholesale funding ratio": "wholesale funding, total liabilities, retail deposits, certificates of deposit, debt securities, interbank market",
    "Assets under custody": "custodian bank, financial market infrastructures, institutional investors, asset servicing, fund administration",
    "Payments cleared and settled through payment systems": "payment processing, clearing house, liquidity providers, financial transactions, central counterparties, real-time gross settlement (RTGS)",
    "Values of underwritten transactions in debt and equity markets": "debt issuance, equity issuance, underwriting, primary markets, syndicated loans, investment banking",
    "OTC derivatives notional value": "over-the-counter derivatives, financial contracts, notional value, swaps, options, forwards, derivatives market",
    "Trading book and available-for-sale securities": "financial securities, trading book, mark-to-market, investment portfolios, bond markets, equity markets"
}

In [ ]:
# Function to classify a snippet into a macro topic
def classify_topic(snippet):
    prompt = f"""
    You are a financial classification assistant. Analyze the following financial snippet and classify it into one of the predefined topics.
    Here are the topics and their defining keywords:

    {topics_keywords}

    Financial Snippet: "{snippet}"

    Return ONLY the topic name from the list.
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error processing snippet: {e}")
        return "Total exposure"

In [ ]:
# Apply function to classify Macro Topics
input_df["Macro Topic"] = input_df["Snippet"].astype(str).apply(classify_topic)

# Display first few rows of the updated DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent,Risk,Macro Topic
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral,Low,Total exposure
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral,High,'Cross-jurisdictional claims'
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral,High,Total exposure
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral,High,This financial snippet does not relate directl...
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral,High,Values of underwritten transactions in debt an...


In [ ]:
# Save the updated DataFrame to a new CSV
output_file_path = "classified_macro_topics.csv"
input_df.to_csv(output_file_path, index=False)

In [ ]:
import pandas as pd

# Load your classified DataFrame
file_path = "classified_macro_topics.csv"
input_df = pd.read_csv(file_path)

# Function to clean the Macro Topic column
def clean_macro_topic(topic):
    # Remove unwanted characters
    cleaned_topic = topic.replace("'", "").replace('"', "").strip()

    # Check if it has more than 10 words
    if len(cleaned_topic.split()) > 10:
        return "Multi-Topic"

    return cleaned_topic

# Apply the cleaning function
input_df["Macro Topic"] = input_df["Macro Topic"].astype(str).apply(clean_macro_topic)


# Display first few rows of the updated DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent,Risk,Macro Topic
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral,Low,Total exposure
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral,High,Cross-jurisdictional claims
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral,High,Total exposure
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral,High,Multi-Topic
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral,High,Values of underwritten transactions in debt an...


In [ ]:
# Save the updated DataFrame to a new CSV
output_file_path = "tqc_UBS_classified_macro_topics.csv"
input_df.to_csv(output_file_path, index=False)

# Section 4: Classification of High Level topics

In [ ]:
# Define the topics and associated keywords
topics_keywords = {
    "Revenue", "Cash", "Equity", "Liability", "Reserve", "Fixed Assets", "Receivables", "Payables", "Profit", "Loss"
}

In [ ]:
# Apply function to classify Macro Topics
input_df["High Level Topic"] = input_df["Snippet"].astype(str).apply(classify_topic)

# Display first few rows of the updated DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent,Risk,Macro Topic,High Level Topic
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral,Low,Total exposure,Equity
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral,High,Cross-jurisdictional claims,Liability
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral,High,Total exposure,'Reserve'
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral,High,Multi-Topic,Loss
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral,High,Values of underwritten transactions in debt an...,Equity


In [ ]:
# Function to clean the Macro Topic column
# Function to clean the Macro Topic column
def clean_macro_topic(topic):
    # Remove unwanted characters
    cleaned_topic = topic.replace("'", "").replace('"', "").strip()

    # Check if it has more than 10 words
    if len(cleaned_topic.split()) > 10:
        return "Multi-Topic"

    return cleaned_topic

# Apply the cleaning function
input_df["High Level Topic"] = input_df["High Level Topic"].astype(str).apply(clean_macro_topic)


# Display first few rows of the updated DataFrame
input_df.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive,Question_topic,Question_topic_description,Topic,Snippet,snippet_sent,Risk,Macro Topic,High Level Topic
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Capital Requirements,okay thank you capital requirements know situa...,Neutral,Low,Total exposure,Equity
1,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Business Scope and Exit,determine scope perimeter activities run parti...,Neutral,High,Cross-jurisdictional claims,Liability
2,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Operational Risk,will course impact op-risk capital charges on,Neutral,High,Total exposure,Reserve
3,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Communication and Clarity,clarity aspects communicate expect something s...,Neutral,High,Multi-Topic,Loss
4,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti'],### Topics Analysis from Financial Text:,1. **Capital Requirements and Scope of Perimet...,Transaction and Value,merits value accretion creation transaction re...,Neutral,High,Values of underwritten transactions in debt an...,Equity


In [ ]:
# Save the updated DataFrame to a new CSV
output_file_path = "tqc_ubs_classified_highlevel_topics.csv"
input_df.to_csv(output_file_path, index=False)